In [1]:
!pip install html2text simpletransformers --quiet

     |████████████████████████████████| 204kB 15.4MB/s 
     |████████████████████████████████| 1.8MB 28.9MB/s 
     |████████████████████████████████| 1.1MB 45.7MB/s 
     |████████████████████████████████| 2.9MB 45.2MB/s 
     |████████████████████████████████| 7.4MB 37.4MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 317kB 48.7MB/s 
     |████████████████████████████████| 1.4MB 41.2MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
     |████████████████████████████████| 133kB 41.9MB/s 
     |████████████████████████████████| 102kB 11.2MB/s 
     |████████████████████████████████| 102kB 11.6MB/s 
     |████████████████████████████████| 163kB 53.1MB/s 
     |████████████████████████████████| 4.5MB 45.6MB/s 
     |████████████████████████████████| 112kB 52.4MB/s 
     |████████████████████████████████| 890kB 47.4MB/s 
     |████████████████████████████████| 71kB 9.7MB/s 
     |████████████████████████████████| 122kB 40.7MB/s

In [14]:
from bs4 import BeautifulSoup
import requests
import html2text
from markdown import markdown
import re
from googlesearch import search
from simpletransformers.question_answering import QuestionAnsweringModel

In [4]:
model = QuestionAnsweringModel('distilbert','distilbert-base-uncased-distilled-squad')

In [125]:
def predict_answer(model, question, contexts, seq_len=256, debug=False):
    split_context = []
    
    if not isinstance(contexts, list):
        contexts = [contexts]
    
    for context in contexts:
        for i in range(0, len(context), seq_len):
            split_context.append(context[i:i+seq_len])
            
    split_context = contexts
    
    f_data = []
    
    for i, c in enumerate(split_context):
        f_data.append(
            {'qas': 
              [{'question': question,
               'id': i,
               'answers': [{'text': ' ', 'answer_start': 0}],
               'is_impossible': False}],
              'context': c
            })
        
    prediction = model.predict(f_data)
    print(type(prediction))
    if debug:
        print(prediction)
    preds = []
    preds = prediction[0][0]['answer']
    
    #preds = [preds[0].lower().strip() for x in prediction if preds[0].strip() != ' ']
    if preds:
      return sorted(list(set(preds)), key=preds.count, reverse=True)
    return 'No answer'

In [130]:
def markdown_to_text(markdown_string):
    """ Converts a markdown string to plaintext """

    # md -> html -> text since BeautifulSoup can extract text cleanly
    html = markdown(markdown_string)

    # remove code snippets
    html = re.sub(r'<pre>(.*?)</pre>', ' ', html)
    html = re.sub(r'<code>(.*?)</code >', ' ', html)

    # extract text
    soup = BeautifulSoup(html, "html.parser")
    text = ''.join(soup.findAll(text=True))

    return text

def format_text(text):
    text = markdown_to_text(text)
    text = text.replace('\n', ' ')
    return text

def query_pages(query, n=5):
    links = list(search(ques, stop=n))
    links = links[::-1][0:6]
    return links

def query_to_text(query, n=5):
    html_conv = html2text.HTML2Text()
    html_conv.ignore_links = True
    html_conv.escape_all = True
    
    text = []
    for link in query_pages(query, n):
        req = requests.get(link)
        text.append(html_conv.handle(req.text))
        text[-1] = format_text(text[-1])
        
    return text

def q_to_a(model, question, n=5, debug=False):
    new_pred = []
    opt = ''
    context = query_to_text(question, n=n)
    pred = list(set(predict_answer(model, question, context, debug=debug)))
    for p in pred:
     new_pred.append(p.capitalize())
     opt += (p.capitalize() + '. ')
    return opt

In [133]:
ques = input('Enter your question here: ')
q_to_a(model, ques, n=8)

Enter your question here: What are the symptoms of COVID


add example index and unique id: 100%|██████████| 6/6 [00:00<00:00, 36419.43it/s]



<class 'tuple'>


'Fever, coughing, difficulty breathing or shortness of breath, lethargy, sneezing, discharge from the nose or eyes, vomiting. Not delay getting emergency care for your health problems or any health condition that requires immediate attention. Signs of sickness in dogs may include fever, coughing, difficulty breathing or shortness of breath, lethargy, sneezing, discharge from the nose or eyes, vomiting,. Coughs and sneezes with a tissue (or use the inside of your elbow. Cover coughs and sneezes. Coughs and sneezes with a tissue (or use the inside of your elbow), throw used tissues in the trash, and wash hands.. Coughs and sneezes. Do not delay getting emergency care for your health problems or any health condition that requires immediate attention.. Coughs and sneezes with a tissue (or use the inside of your elbow). Vomiting. Trouble breathing, persistent pain or pressure in the. Coughs and sneezes with a tissue (or use the inside of your elbow), throw used tissues in the trash, and was